In [16]:
import os
import json
with open('./.kaggle/kaggle.json') as credentials_file:
    credentials_dict = json.load(credentials_file)
    os.environ['KAGGLE_USERNAME'] = credentials_dict['username']
    os.environ['KAGGLE_KEY'] = credentials_dict['key']
    import kaggle
from zipfile import ZipFile
import pandas as pd
import tensorflow as tf
import keras
from tqdm import tqdm

In [8]:
kaggle.api.competition_download_files(competition='store-sales-time-series-forecasting', path='./dataset', force=False, quiet=True)
with ZipFile('./dataset/store-sales-time-series-forecasting.zip') as dataset_zip:
    dataset_zip.extractall('./dataset')

In [ ]:
# The idea of this competition, is to predict sales for mnay product families for stores in ecuador.

In [33]:
#Train and test share: date, store_nbr, family(product_family), onpromotion
#Train also has the sales.
#Test is what we are trying to predict in the competition
train_df = pd.read_csv('./dataset/train.csv', index_col='id')
test_df = pd.read_csv('./dataset/test.csv', index_col='id')
#Stores contains store metadata, we might want to join that data into our train and test tables to improve predictions if they are not redundant with the store_nbr (aka if at least some of the stores share their values).
stores_df = pd.read_csv('./dataset/stores.csv', index_col='store_nbr') #stores metadata

#Daily oil price, keep in mind oil price affects ecuador economical health.
#We probably want to join it into our dataframe, but keep in mind it seems to have some null values. We probably want to fill downwards so that it takes its value from the previous date but well have to see. We could also average between the next and previous day.
oil_df = pd.read_csv('./dataset/oil.csv') 

transactions_df = pd.read_csv('./dataset/transactions.csv')

#A table containing holiday events. 
#its worth noting that some events were transfered in date so we need to take into account hte actual date.
#Also, keep in mind that some seem to be regional and some local. This could mean that we need to take into account the region in considering them.
#Some holidays were of type bridge, meaning  they are extra days added to the same holiday.
#This are often compensated by making some not working days(ie saturday) working days. This are of type workday.
holiday_events_df = pd.read_csv('./dataset/holidays_events.csv')
sample_submission_df = pd.read_csv('./dataset/sample_submission.csv')

#Additonally to these tables wages on the public sector are paid on the 15th and last day of each month which could affect supermarket sales
#A great earthquiake struck ecuador on april 16 2016 which resulted in donations possibly affecting sales.


#TLDR we need to add many features to the train and test df.
#First lets make some column names easier to understand. Specially after joining.
train_df = train_df.rename(columns={'family':'product_family'})
test_df = test_df.rename(columns={'family':'product_family'})
oil_df = oil_df.rename(columns={'dcoilwtico':'oil_price'})
stores_df = stores_df.rename(columns={'transactions':'all_products_transactions'})
holiday_events_df = holiday_events_df.rename(columns={'type':'day_type', 'locale':'holiday_locale_type', 'locale_name':'holiday_locale','description':'holiday_description'})

In [32]:
display(train_df.head(3))
display(test_df.head(3))
display(stores_df.head(3))
display(oil_df.head(3))
display(holiday_events_df.head(3))
display(transactions_df.head(3))

,date,store_nbr,product_family,sales,onpromotion
id,,,,,
0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,2013-01-01,1,BABY CARE,0.0,0
2,2013-01-01,1,BEAUTY,0.0,0


,date,store_nbr,product_family,onpromotion
id,,,,
3000888,2017-08-16,1,AUTOMOTIVE,0
3000889,2017-08-16,1,BABY CARE,0
3000890,2017-08-16,1,BEAUTY,2


,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8


,date,oil_price
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358


In [ ]:
def add_columns(features_df:pd.DataFrame):
    full_features_df = features_df.join(oil_df,on='date',how='left')
    full_features_df = full_features_df.join(stores_df, on=['date','store_nbr'], how='left') #Need to also consider date.
    full_features_df = full_features_df.join(holiday_events_df, on='date')
    full_features_df[]

    return features_df